In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.1 MB/s eta 0:00:00


## Tokenizer

In [19]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [26]:
raw_inputs = (['This food is really good.', "I don't like how it tastes."])
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='tf')
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[  101,  2023,  2833,  2003,  2428,  2204,  1012,   102,     0,
            0,     0],
       [  101,  1045,  2123,  1005,  1056,  2066,  2129,  2009, 16958,
         1012,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 11), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


## Model

In [27]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_197']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Output

In [28]:
outputs = model(inputs)
print(outputs.logits.shape)

(2, 2)


## Postprocessing the output

In [29]:
print(outputs.logits)

tf.Tensor(
[[-4.2772193  4.6635175]
 [ 3.4940817 -2.8776333]], shape=(2, 2), dtype=float32)


Those are not probabilities but logits, the raw, unnormalized scores outputted by the last layer of the model. To be converted to probabilities, they need to go through a SoftMax layer (all 🤗 Transformers models output the logits, as the loss function for training will generally fuse the last activation function, such as SoftMax, with the actual loss function, such as cross entropy):

In [30]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[1.3092738e-04 9.9986911e-01]
 [9.9829370e-01 1.7063088e-03]], shape=(2, 2), dtype=float32)


To get the labels corresponding to each position, we can inspect the id2label attribute of the model config 

In [31]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}